<div align="center"><b><font size=6>ORTA - Group Project Report<b></div>
<div align="center"><font size=6>Dial-a-ride problem (DARP) with tranfers</div>

<div align="center"><font size=5>Group's members: s313107, s319876, s332315</div>

## <ins>Introduction</ins>

The main scope of this project is to solve the DARP with Transfers (DARPT), focusing on the choice and implementation of a heuristic. Since this problem has been studied in-depth and it is well-documented in literature, the mathematical modelling was skipped, concentrating instead on a precise formalization of our case of interest.

## <ins>Problem Formalization</ins>

First of all, it has to be specified that our approach for solving the DARPT is an offline one, where our a-priori knowledge of the scenario we are operating into is complete in all its aspects.

Generating an instance of our DARPT formalization regards generating specific information on three attributes of our problem:
- Passengers to be picked-up and dropped to their destination
- Transfers with their capacities and their dedicated depot
- Map of the scenario, modelled as an undirected graph

Following this order, we are now going to see thoroughly how each one of these attributes is characterized. The code regarding the generation of the whole instance has to be found in "instance_gen" folder.


### <ins>1. Passengers</ins>

A passenger is characterized by three parameters:
- 'start_position' as the position in which the passenger will be picked-up by the transfer
- 'stop_position' as the final destination of the passenger, where it will be dropped by the transfer
- 'time_request' as a non-mandatory condition on the preferred time of arrival of the passenger

### <ins>2. Transfers</ins>
 A transfer is charachterized, as a passenger, by three parameters:
 - 'start_position' as the starting depot for the path of the transfer
 - 'stop_position' as the stopping depot for the path of the trasnfer, once it has dropped all the carried passengers
 - 'capacity' as the maximum amount of people that can be carried at the same time


<ins>N.B.</ins> It is important to specify that the data structures chosen for the representation of both passengers and transfers is that of dictionaries

### <ins>3. Map of the scenario</ins>

In [ ]:
# Auto-reload modules for each new run
%reload_ext autoreload
# Imports 
import networkx as nx
import matplotlib.pyplot as plt
from instance_gen.instances import createGraphInstance

# Create graph according to user
G = createGraphInstance(6,1,10,20,100,2)

# Position the nodes using a layout
pos = nx.spring_layout(G)

# Draw the graph
plt.figure()
nx.draw_networkx(G, pos, with_labels=True, node_size=700, node_color='lightblue', font_size=10, font_weight='bold')

# Prepare edge labels with both weights
edge_labels = {(u, v): f'({d["fuel_cost"]}, {d["time_cost"]})' for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Display the plot with informative box
plt.title('Graph of the current scenario')
plt.show()

## <ins>Resolution method</ins>

### The choice of the heuristic : the Genetic algorithm

### Generation processes

#### 1 - The initial population

To generate the initial population, we state that it is sufficient to provide a "good enough" pool of individuals to start with, given that the algorithm will improve them through the generations. 
By "good enough", we mean that each individual/solution satisfies the constraints of the problem but could be far from optimal. DO WE HAVE TO SPECIFY (MAYBE BEFORE) WHAT INDIVIDUALS OR POPULATION ARE?

Therefore, we generate the order of passengers to retrieve at random for each vehicle, ensuring that this latter is not overloaded. 

Given a number of passengers (for example 4) and a list of capacities (for example [1,2,3] meaning that vehicle 1 has capacity of 1, vehicle 2 has capacity of 2 and vehicle 3 has capacity of 3) the generation function returns a list of as many sublists as vehicles, of sizes the capacity of the vehicles. Each sublist is randomly filled with numbers from 1 to the number of passengers but two sublists don't contain the same number (one passenger is only fetched by one vehicle). If they are not enough passengers for the sublists to be the size of the capacities, they are filled with zeros. FOR EXAMPLE: I would have [ [ 1 ], [ 3, 0 ], [ 2, 4, 0] ]. The zeros mean that one seat is not taken. 

#### 2 - Fitness

To compare two individuals, we need some accurate measurements of "how good" one solution is. 

To accomplish this task, a Fitness function, where we calcuate the overall cost of a solution, has been develloped.  By overall cost, we mean the linear combination of the fuel cost of a path and the time cost related to that path, that is $ overall\_  cost = w_f * fuel\_ cost + w_t * time\_ cost$, where $w_f$ and $w_t$ are two weights, respectively fuel weight and time weight, that can be used to decide which aspect to privilege during optimization: the fuel consumed by the transfers, or the time to complete each path. This allows to optimize the route that each vehicle must follow to pick up and drop each passenger requesting the service, considering both the needs of the company, so, of the transfers, and of the passengers, which, as specified above, have a time request which if not respected arises a penalty, which has effects on the time cost:  $time\_ cost = path\_time\_ cost + time\_ penalty$.
Obviously, to compute all the required quantities, the knowledge of the path of each transfer, given the passengers assigned to them, is needed. Thus, a Routing Algorithm is necessary for this task. The flow of the routing algorithm is the following:
- Starting from the transfer source node (starting depot), calculate the minimum length path to the first passenger to pick up's node ("start_position");
- Add the found path to the list containing the whole transfer path;
- In a loop, until we get the last drop node (last passenger's "stop_position"):
     - find the minimum length path between the current node transfer is and the destination node of the first passenger which must be dropped;
     - find the minimum length path between the current node transfer is and the starting node of the next passenger which must be picked up;
     - choose the minimum between the two previously calculated paths;
     - add the found path to the list containing the whole transfer path;
     - update the current node with the one we are in.;
     - if the current node corresponds to the last "stop_position" of the passengers, break the loop.
- Find the minimum length path between the current node transfer is and the destination node of the transfer (stopping depot);
- Add the found path to the list containing the whole transfer path;

All the minimum length paths are calculated using the Dijkstra's algorithm, already implemented in the networkx library, and using as weights, the ones assigned on each edge of the graph.
Using this routing algorithm, all the minimum paths for each transfer have been found, and now it is possible to calculate the $fuel\_ cost$, the $time\_ cost$ and, in case, the penalties too. COMMENTARE LA SUBOTTIMALITà 

#### 3 - Selection, crossover and mutation

<b>Selection</b>

The selection process aims at selecting the most "fitted" individuals for the production of the offsprings. "Most fitted" doesn't necessarily means the ones with the best fitness because systematically keeping the best individuals would deteriorate the genetic diversity of the algorithm and we would converge towards a local optimum much more faster and thus not explore the solution space enough. 

We implemented two selection methods : the roulette wheel and the tournament selections. Both rely on the fitness but introduce different concurrency schemes. 

For the roulette wheel, we imagine a wheel on which each portion corresponds to an individual and is proportional to its quality - meaning its fitness. The better the individual, the bigger its portion on the wheel is, the more chances it has to be selectioned at each round. 
In practice, we associate each individual to a probability of getting picked and sort each individual according to them. Then, we simulate a random draw and get the desired number of individuals. 
This method is quite adapted to selection the parents because she favors the more performant individuals (the best solutions) but also leaves some chances for the less good solution to be picked. The solution space is thus not limited. 

For the tournament selection, the individuals are still selected based on their fitness but through a tournament process that runs as many times as they are individuals in the population. In each tournament, a given number of individuals are randomly selected from the population and the one with the highest fitness is selected as the winner. It is then added to a list which size should match the population size at the end of the process. 

The advantage of this approach is that it introduces selection pressure, where fitter individuals are more likely to be selected, while maintaining a diverse population by allowing random sampling and preventing duplicates. 

<b>Crossover and mutation</b>

The selectioned individuals are then used to generate new offsprings thanks to two operators : the crossover and mutation. 

For the crossover, we combine parts of two parent solutions to create a new child solution, ensuring that this latter maintains the structure and constrains of the parents. 
Two random crossver points are chosen. A slice from the first parent between the crossover points is copied to the corresonding position in the child list and the remaining slots are filled with the remaining elements from the second parent that are not already in the child. 

If we operate in a "systematic" manner on our solutions, we take the risk of converging towards a local optimum and thus not to sufficiently explore the space domain. Therefore, we introduce some randomness by modifying the genetic characteristic of an individual with a mutation operator. Even if the outcoming solutions are not better, they allow us to explore a bit more the solution space by occasionally making random changes to the solutions. 

Our mutation function introduces variability into a solution by randomly shuffling a part of it based on a given probability. 

## <ins>Results</ins>

#### Key observations

Infine, our genetic algorithm is completely functional. One great clue is the best fitness over the generation process. 

[ PLOT 1 ]

The curve shows a rapid decrease initially, indicating that the genetic algorithm quickly finds better solutions. After a few generations, the curve flattends out, suggesting that the algorithm has found a stable and potentially optimal solution that does not improve much further. 

Another consistent indicator to evaluate the performance of our algorithm is the evolution of the mean fitness of the top first individuals. 

[ PLOT 2]

This curve also shows a sharp initial decline, similar to the previous curve, indicating that the top 5 individuals are improving significantly in the early generations. 
The curve then becomes less steep and shows some fluctuations, which might indicate ongoing exploration and slight improvements in the top solutions. 
There is a general trend of improvement over generations, but the curve is less smooth than the best fitness curve, reflecting variations in the selection of the top individuals. 


#### Performance analysis

To have a more accurate opinion of our performance, we can compare the evolution of 3 main indicators. 
The following plots show the evolution of the fitness values over 100 generations for a population of 20 individuals. 

<span style="color:blue">The blue curve represents the evolution of the best fitness value in the population for each generation. </span>

<span style="color:red">The red curve represents the mean fitness of the top 5 best individuals in each generation. </span>

<span style="color:green">The green curve represents the mean fitness of the entire population for each generation. </span>

- First, we notice that at the beginning, the fitness values are quite high which is expected as the initial population is randomly generated, leading to suboptimal solutions. 

- <b> Convergence : </b>The algorithm demonstrates good convergence, as indicated by the rapid improvement in the best fitness values. The flattening of the blue curve suggests that the algorithm has converged to a solution that it considers optimal. 

- <b> Exploration VS exploitation : </b> The fluctuations in the red and green curves indicate a balance between exploration (searching new areas of the solution space) and exploitation (refining known good solutions); 

- <b> Diversity : </b> The difference between the blue curve (best fitness) and the green curve (mean fitness) indicates that there is still some diversity in the population, which is good for avoiding local optima. The gap between the best fitness and the mean fitness suggests that while the best solutions are very good, the overall population still has a range of fitness values, promoting continued evolution. 

- <b> Effectiveness : </b> The sharp initial decline in fitness values demonstrates the effectiveness of the genetic operators in finding better solutions quickly and the continuous improvement indicates that the algorithm is effective in maintaining and promoting high-quality solutions. 


To conclude, the genetic algorithm is performing well, as evidenced by the rapid convergence of the fitness values and the ongoing improvement. It also maintains a good balance between exploration and exploitation, ensuring continued evolution without the risk of premature convergence. The diversity feature is very positive, allowing the algorithm to escape local optima. 

To further improve it, we could consider some of the following solutions : 
1. Adjusting furthermore the operator parameters, tuning mutation and crossover rates to improve even more the balance between exploration and exploitation. 

2. Increase the population size, if it is computationally feasible and doesn't increase the execution time too much. It is already taking a few minutes for a small population of 20 individuals. A larger population might improve diversity and exploration of the solution space.  


![Progression of the solutions quality](plot1.png)

In [4]:
import IPython
print(IPython.__version__)

8.19.0


In [8]:
from IPython.display import Image, display
import os

# Check if the file exists
file_path = 'plot1.png'
if os.path.exists(file_path):
    # Create an image object
    img = Image(filename=file_path)
    # Display the image
    display(img)
else:
    print(f"File '{file_path}' does not exist.")

## <ins>Conclusion</ins>

### Comments and further implementation

Looking back at what we did, a major improvement we could think of is the construction of one solution. The current modelization being very simple allows us to efficiently solve the problem without too much computational effort. However, looking at one solution, we can't really tell the path taken by each vehicle. 
For example, let's say that one solution suggests that vehicle 1 takes 4 passenger in this order : 1, 2, 3 and 4. We can't tell if the vehicle first takes passenger 1, then 2, then drops the 1 or if he takes passenger 1, drops him and then takes passenger 2. 
To have a clearer view, a post processing is necessary, involving an algorithm to find the shortest path given a starting point, a destination point, and several locations to stop by. 
Another alternative would have been to directly design a solution as a list of positions instead of a list of individuals. It would be more intuitive to the user and it would be easy to retrieve the passengers taken by each vehicle and store them in memory. 

At the fitness level, we think the current implementation is very satisfying. Not only it takes the time and fuel costs into account, but also the quality of the service by computing the delay for each passenger, regarding their desired arrival time. According to the priority, the weight of each factor can be modified to favor one objective : either minimizing the cost, or maximizing the customer satisfaction. 

Concerning the computational effort, it is rather reasonable for such a problem. It takes about 3 minutes to solve with 100 generations and 10 minutes with 500 generations. As we are already quite satisfied with the outcome with 100 generations, such a wait shouldn't be needed for most problems we could encounter. 